In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os

### read 'survey_standard.csv' data

In [2]:
fname = 'survey_standard_2021-03-22.csv'

df_raw = pd.read_csv(os.path.join('M:\\Data\\OnBoard\\Data and Reports\\_data Standardized', fname), encoding = 'ISO-8859-1')

C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,7,9,22,23,26,27,28,29,30,32,40,41,42,52,55,56) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### create summary table of non-spatial variables for Tableau visualization

In [3]:
# create field "operator_survey_year" which will be used as a Tableau filter

df = df_raw.copy()
df['operator_survey_year'] = df['operator'] + ' - ' + df['survey_year'].apply(lambda x: str(x))

print('Read {} rows from the data including the following surveys: \n{}'.format(df.shape[0],
                                                                              list(df.operator_survey_year.unique())))

display(df.head())

Read 146457 rows from the data including the following surveys: 
['AC Transit - 2018', 'ACE - 2019', 'BART - 2015', 'Caltrain - 2014', 'Capitol Corridor - 2019', 'City Coach - 2017', 'County Connection - 2019', 'Delta Breeze - 2017', 'FAST - 2017', 'Golden Gate Transit - 2018', 'LAVTA - 2018', 'Marin Transit - 2017', 'Napa Vine - 2014', 'Napa Vine - 2019', 'Petaluma Transit - 2018', 'Santa Rosa CityBus - 2018', 'SF Muni - 2017', 'Soltrans - 2017', 'Sonoma-Marin Area Rail Transit - 2018', 'Sonoma County Transit - 2018', 'TriDelta - 2019', 'Union City Transit - 2017', 'VTA - 2017', 'WestCAT - 2017', 'WETA - 2019']


,ID,operator,survey_year,survey_tech,access_mode,alt_weight,depart_hour,depart_time,dest_purp,direction,...,home_maz,orig_maz,school_maz,workplace_maz,board_tap,alight_tap,trip_weight,field_language,survey_time,operator_survey_year
0,1,AC Transit,2018,local bus,walk,NaN,NaN,NaN,work,NaN,...,11514.0,11514.0,NaN,425106.0,90131.0,490370.0,1.855557,ENGLISH,17:30:00,AC Transit - 2018
1,10,AC Transit,2018,local bus,walk,NaN,NaN,NaN,work,NaN,...,316876.0,316876.0,NaN,324992.0,390602.0,390316.0,4.847912,ENGLISH,07:30:00,AC Transit - 2018
2,100,AC Transit,2018,local bus,walk,NaN,NaN,NaN,home,NaN,...,318470.0,318567.0,NaN,318567.0,390526.0,390132.0,9.772444,ENGLISH,17:30:00,AC Transit - 2018
3,1000,AC Transit,2018,local bus,walk,NaN,NaN,NaN,work,NaN,...,327175.0,327175.0,NaN,325899.0,390316.0,390746.0,10.572424,ENGLISH,08:30:00,AC Transit - 2018
4,10000,AC Transit,2018,local bus,walk,NaN,7.0,NaN,work-related,NaN,...,324492.0,323825.0,NaN,311933.0,390086.0,390449.0,1.270000,ENGLISH,11:30:00,AC Transit - 2018


In [ ]:
# consistently label 'missing' value for fields that should not have na
missing_value_dict = {'missing': 'missing',
                      'MISSING': 'missing',
                      'Missing': 'missing'}

for colname in ['race', 'hispanic', 'household_income', 'approximate_age', 'work_status', 'student_status', 'auto_suff',
                'access_mode', 'egress_mode', 'boardings',  'depart_hour', 'return_hour',
                'tour_purp', 'weekpart', 'day_part', 
                'fare_medium', 'fare_category', 'eng_proficient',
                'persons', 'gender', 'worker_numeric_cat', 'vehicle_numeric_cat']:
    df[colname+'_temp'] = df[colname].map(missing_value_dict)
    df.loc[df[colname+'_temp'] == 'missing', colname] = 'missing'
    df.loc[df[colname].isnull(), colname] = 'missing'

    missing = df.loc[df[colname] == 'missing']
    print('{} has missing value in {} rows, {:.1%} of total'.format(colname, missing.shape[0], missing.shape[0]/df.shape[0]))
    
    df.drop(columns = [colname+'_temp'], inplace=True)
    

# 'work_status' and 'student_status' fields can have na, so not 'missing'
for i in ['work_status', 'student_status']:
    df.loc[df[i] == 'missing', i] = np.nan

race has missing value in 0 rows, 0.0% of total
hispanic has missing value in 4615 rows, 3.2% of total
household_income has missing value in 30497 rows, 20.8% of total
approximate_age has missing value in 7659 rows, 5.2% of total


In [ ]:
# summarize access and egress modes
df['access_egress_modes'] = df['access_mode'] + '-' + df['egress_mode']
# display(df.access_egress_modes.value_counts())

access_egress_mode_dict = {'walk-walk': 'walk at both ends',
                           'pnr-walk': 'pnr at one end',
                           'knr-walk': 'knr at one end',
                           'walk-pnr': 'pnr at one end',
                           'walk-knr': 'knr at one end',
                           'bike-bike': 'bike at both ends',
                           'missing-missing': 'missing at least one end',
                           'pnr-pnr': 'pnr at both ends',
                           'knr-knr': 'knr at both ends',
                           'knr-pnr': 'pnr and knr',
                           'pnr-knr': 'pnr and knr',
                           'bike-walk': 'bike at one end',
                           'walk-bike': 'bike at one end',
                           'tnc-walk': 'tnc at one end',
                           'walk-tnc': 'tnc at one end',
                           'bike-pnr': 'pnr at one end',
                           'pnr-bike': 'pnr at one end',
                           'bike-knr': 'knr at one end',
                           'knr-bike': 'knr at one end',
                           'walk-missing': 'missing at least one end',
                           'missing-walk': 'missing at least one end',
                           'other-walk': 'other',
                           'tnc-tnc': 'tnc at both ends',
                           'pnr-tnc': 'pnr and tnc',
                           'knr-tnc': 'pnr and tnc',
                           'tnc-knr': 'pnr and tnc',
                           'tnc-pnr': 'pnr and tnc',
                           'walk-other': 'other',
                           'missing-pnr': 'missing at least one end',
                           'pnr-missing': 'missing at least one end',
                           'missing-knr': 'missing at least one end',
                           'bike-tnc': 'tnc at one end',
                           'knr-missing': 'missing at least one end',
                           'bike-missing': 'missing at least one end',
                           'tnc-bike': 'tnc at one end',
                           'knr-other': 'knr at one end',
                           'other-other': 'other',
                           'other-pnr': 'pnr at one end',
                           'other-knr': 'knr at one end',
                           'other-bike': 'bike at one end',
                           'missing-tnc': 'missing at least one end',
                           'missing-other':'missing at least one end',
                           'tnc-missing': 'missing at least one end'}

df['access_egress_modes'] = df['access_egress_modes'].map(access_egress_mode_dict)
display(df.access_egress_modes.value_counts())

In [ ]:
# create 'age_group' field

df.loc[df.approximate_age == 'missing', 'approximate_age'] = np.nan
df['approximate_age'] = df['approximate_age'].fillna(0)
df['approximate_age'] = df['approximate_age'].apply(lambda x: int(x))
df['age_group'] = 'missing'
df.loc[(df.approximate_age > 0) & (df.approximate_age < 16), 'age_group'] = 'below than 16'
df.loc[(df.approximate_age > 15) & (df.approximate_age < 23), 'age_group'] = '16 to 22'
df.loc[(df.approximate_age > 22) & (df.approximate_age < 30), 'age_group'] = '23 to 29'
df.loc[(df.approximate_age > 29) & (df.approximate_age < 40), 'age_group'] = '30 to 39'
df.loc[(df.approximate_age > 39) & (df.approximate_age < 50), 'age_group'] = '40 to 49'
df.loc[(df.approximate_age > 49) & (df.approximate_age < 60), 'age_group'] = '50 to 59'
df.loc[(df.approximate_age > 59) & (df.approximate_age < 70), 'age_group'] = '60 to 69'
df.loc[df.approximate_age > 69, 'age_group'] = 'Above 69'
display(df.age_group.value_counts())

In [ ]:
# summarize fare medium and fare category

# use standard value dictionary
dict_df = pd.read_csv(r'C:\Users\ywang\Documents\GitHub\onboard-surveys\util\standard_variable_dict.csv',
                      usecols = ['generic_variable',
                                 'valid_values_for_categoric_variables',
                                 'standard_values_for_categoric_variables'])
dict_df.columns = ['variable_name','value_details','value_summary']

for colname in ['fare_medium', 'fare_category']:
    df[colname] = df[colname].apply(lambda x: x.strip())
    value_dict = dict_df.loc[dict_df.variable_name == colname]
    df = df.merge(value_dict, left_on=colname, right_on='value_details', how='left')
    df.rename(columns = {'value_summary': colname+'_summary'}, inplace=True)
    display(df[colname+'_summary'].value_counts())

In [ ]:
# select fields for visualization
basic_info = ['operator_survey_year', 'weekpart', 'day_part', 'trip_weight', 'weight']

trip_info = ['access_egress_modes', 'access_mode', 'egress_mode', 'tour_purp', 'boardings',
             'fare_category_summary', 'fare_medium_summary',
             'commuter_rail_present', 'heavy_rail_present', 'ferry_present', 'light_rail_present', 'express_bus_present']

demo_info = ['persons', 'work_status', 'student_status', 'age_group', 'gender', 'race', 'hispanic', 'eng_proficient', 
             'household_income', 'worker_numeric_cat', 'vehicle_numeric_cat', 'auto_suff']

spatial_info = ['orig_taz', 'dest_taz', 'home_taz', 'workplace_taz', 'school_taz',
#                 'orig_maz', 'dest_maz', 'home_maz', 'workplace_maz', 'school_maz',
                'first_board_lat', 'first_board_lon', 'last_alight_lat', 'last_alight_lon']


df_summary = df[basic_info + trip_info + demo_info + spatial_info]

# check NA in columns
for i in list(df_summary):
    if df_summary[i].isnull().values.any():
        print('column with NA: {}'.format(i))
    if 'missing' in (df_summary[i].unique()):
        print('column with missing: {}'.format(i))

In [ ]:
# export
new_fname = fname.split('.')[0] + '_forTableau.csv'
df_summary.to_csv(os.path.join('M:\\Data\\OnBoard\\Data and Reports\\_data Standardized', new_fname), index=False)

### create TAZ-level spatial data

In [ ]:
taz_df = gpd.read_file(r'M:\Data\OnBoard\Data and Reports\_geocoding Standardized\TM2_Zones\tazs.shp')

In [ ]:
# convert taz data to integer
for i in ['orig_taz', 'dest_taz', 'home_taz', 'workplace_taz', 'school_taz']:
    df_summary[i] = df_summary[i].fillna(0)
    df_summary[i] = df_summary[i].apply(lambda x: int(x))

In [ ]:
orig_taz = df_summary.groupby(['operator_survey_year', 'weekpart', 'day_part', 'orig_taz'])['trip_weight'].sum().reset_index()
orig_taz.rename(columns = {'trip_weight': 'trip_weight_by_orig_taz',
                           'orig_taz': 'TM2_taz'}, inplace=True)
# print(orig_taz.shape)
# print(list(orig_taz))

dest_taz = df_summary.groupby(['operator_survey_year', 'weekpart', 'day_part', 'dest_taz'])['trip_weight'].sum().reset_index()
dest_taz.rename(columns = {'trip_weight': 'trip_weight_by_dest_taz',
                           'dest_taz': 'TM2_taz'}, inplace=True)
# print(dest_taz.shape)
# print(list(dest_taz))

home_taz = df_summary.groupby(['operator_survey_year', 'weekpart', 'day_part', 'home_taz'])['trip_weight'].sum().reset_index()
home_taz.rename(columns = {'trip_weight': 'trip_weight_by_home_taz',
                           'home_taz': 'TM2_taz'}, inplace=True)
# print(home_taz.shape)
# print(list(home_taz))

workplace_taz = df_summary.groupby(['operator_survey_year', 'weekpart', 'day_part', 'workplace_taz'])['trip_weight'].sum().reset_index()
workplace_taz.rename(columns = {'trip_weight': 'trip_weight_by_workplace_taz',
                                'workplace_taz': 'TM2_taz'}, inplace=True)
# print(workplace_taz.shape)
# print(list(workplace_taz))

school_taz = df_summary.groupby(['operator_survey_year', 'weekpart', 'day_part', 'school_taz'])['trip_weight'].sum().reset_index()
school_taz.rename(columns = {'trip_weight': 'trip_weight_by_school_taz',
                             'school_taz': 'TM2_taz'}, inplace=True)
# print(school_taz.shape)
# print(list(school_taz))

In [ ]:
# merge
TM2_taz_summary = orig_taz.merge(dest_taz,
                                 on=['operator_survey_year', 'weekpart', 'day_part', 'TM2_taz'],
                                 how='outer').merge(home_taz,
                                                    on=['operator_survey_year', 'weekpart', 'day_part', 'TM2_taz'],
                                                    how='outer').merge(workplace_taz,
                                                                       on=['operator_survey_year', 'weekpart', 'day_part', 'TM2_taz'],
                                                                       how='outer').merge(school_taz,
                                                                                          on=['operator_survey_year', 'weekpart', 'day_part', 'TM2_taz'],
                                                                                          how='outer')
# fill 'na' in trip_weights_sum with 0
for i in ['trip_weight_by_orig_taz', 'trip_weight_by_dest_taz',
          'trip_weight_by_home_taz', 'trip_weight_by_workplace_taz', 'trip_weight_by_school_taz']:
#     print(TM2_taz_summary.loc[TM2_taz_summary[i].isnull()].shape[0])
    TM2_taz_summary[i] = TM2_taz_summary[i].fillna(0)
#     print(TM2_taz_summary.loc[TM2_taz_summary[i].isnull()].shape[0])

In [ ]:
# export
taz_fname = fname.split('.')[0] + '_TM2_taz_summary.csv'
TM2_taz_summary.to_csv((os.path.join('M:\\Data\\OnBoard\\Data and Reports\\_data Standardized', taz_fname)), index=False)